#ESE 546 Recitation (Sep 11)

### Outline

* Basics in PyTorch
* Autograd
* Building blocks

##Setting up Colab
####Authentication
Initial setup to allow colab to mount user's google drive account.  
Remember to change Runtime type to GPU Hardware accelrated for leveraging GPU resources.
```
> Runtime > Change runtime type > Hardware accelrator > Select GPU
```

####Access to files

In [ ]:
# bootstrap environment into place
from google.colab import auth
import datetime
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
import os
from googleapiclient.http import MediaIoBaseDownload

# mount the user's google drive into the system
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Introduction

In this tutorial, we will cover:

* What is PyTorch : [Ref](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html)
* Autograd: Automatic Differentiation : [Ref](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py)
* Building blocks of Neural Networks : [Ref](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py)


What is PyTorch?
================

It’s a Python-based scientific computing package targeted at two sets of
audiences:

-  A replacement for NumPy to use the power of GPUs
-  A deep learning research platform that provides maximum flexibility
   and speed

Getting Started
---------------

###Tensors

Tensors are similar to NumPy’s ndarrays which can also be used on a GPU to accelerate computing.



In [ ]:
from __future__ import print_function
import torch

Construct a 5 x 3 matrix, uninitialized:



In [ ]:
x = torch.empty(5, 3)
print(x)

tensor([[1.0856e+25, 0.0000e+00, 4.4842e-44],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.0625e+21, 1.6877e-07, 1.7249e-07],
        [4.1532e-08, 2.0921e+23, 4.2464e-08],
        [4.1915e+21, 1.0606e-08, 0.0000e+00]])


Construct a randomly initialized matrix:



In [ ]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8636, 0.7312, 0.7845],
        [0.4756, 0.8539, 0.2228],
        [0.4793, 0.6064, 0.7546],
        [0.5832, 0.3295, 0.1543],
        [0.5180, 0.7827, 0.9301]])


Construct a matrix filled zeros and of dtype long:



In [ ]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


Construct a tensor directly from data:



In [ ]:
x = torch.tensor([5.5, 3])
print(x)

tensor([5.5000, 3.0000])


or create a tensor based on an existing tensor. These methods  
will reuse properties of the input tensor, e.g. dtype, unless  
new values are provided by user



In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.5472,  1.0823,  1.2578],
        [ 0.1609, -0.8441,  0.1895],
        [ 1.4960,  1.2178, -0.3504],
        [ 0.0651, -0.2766,  0.1261],
        [ 0.7235, -0.7144, -0.2124]])


Get its size:

Note: torch.Size is in fact a tuple, so it supports all tuple operations.




In [ ]:
print(x.size())

torch.Size([5, 3])


###Operations
There are multiple syntaxes for operations. In the following  
example, we will take a look at the addition operation.

Addition: syntax 1



In [ ]:
y = torch.rand(5, 3)
print(x + y)

tensor([[ 1.1412,  1.2578,  1.2784],
        [ 0.3775, -0.7161,  0.6354],
        [ 1.8884,  1.3302, -0.1605],
        [ 0.7183,  0.0094,  0.2335],
        [ 1.5238, -0.0575,  0.2156]])


Addition: syntax 2



In [ ]:
print(torch.add(x, y))

tensor([[ 1.1412,  1.2578,  1.2784],
        [ 0.3775, -0.7161,  0.6354],
        [ 1.8884,  1.3302, -0.1605],
        [ 0.7183,  0.0094,  0.2335],
        [ 1.5238, -0.0575,  0.2156]])


Addition: providing an output tensor as argument



In [ ]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[ 1.1412,  1.2578,  1.2784],
        [ 0.3775, -0.7161,  0.6354],
        [ 1.8884,  1.3302, -0.1605],
        [ 0.7183,  0.0094,  0.2335],
        [ 1.5238, -0.0575,  0.2156]])


Addition: in-place



In [ ]:
# adds x to y
y.add_(x) # _ at end means the operation mutates tensor y in-place
print(y)

tensor([[ 1.1412,  1.2578,  1.2784],
        [ 0.3775, -0.7161,  0.6354],
        [ 1.8884,  1.3302, -0.1605],
        [ 0.7183,  0.0094,  0.2335],
        [ 1.5238, -0.0575,  0.2156]])


Indexing: You can use standard NumPy-like indexing with all bells and whistles!



In [ ]:
print(x[:, 1])

tensor([ 1.0823, -0.8441,  1.2178, -0.2766, -0.7144])


Resizing: If you want to resize/reshape tensor, you can use ``torch.view``:



In [ ]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


Tensor2Number: If you have a one element tensor, use ``.item()`` to get the value as a
Python number



In [ ]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([0.0185])
0.018504910171031952


**Read later:**


  100+ Tensor operations, including transposing, indexing, slicing,  
  mathematical operations, linear algebra, random numbers, etc.,  
  are described in the
  [documentation](http://pytorch.org/docs/torch).

NumPy Bridge
------------

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory  
locations, and changing one will change the other.

###Converting a Torch Tensor to a NumPy Array



In [ ]:
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [ ]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


See how the numpy array changed in value.



In [ ]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


###Converting NumPy Array to Torch Tensor
See how changing the np array changed the Torch Tensor automatically



In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


All the Tensors on the CPU except a CharTensor support converting to
NumPy and back.

CUDA Tensors
------------

Tensors can be moved onto any device using the ``.to`` method.



In [ ]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([1.0185], device='cuda:0')
tensor([1.0185], dtype=torch.float64)



Autograd: Automatic Differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.  
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True``, it starts to track all operations on it. When  
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be  
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with `requires_grad=True`,
but for which we don't need the gradients.

There’s one more class which is very important for autograd implementation - a ``Function``.  

<img src="https://miro.medium.com/max/1536/1*wE1f2i7L8QRw8iuVx5mOpw.png" alt="Function" width="600"/>

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has  
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their  
``grad_fn is None``).

<img src="https://miro.medium.com/max/1684/1*FDL9Se9otGzz83F3rofQuA.png" alt="Computation Graph" width="500"/>

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element  
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``  
argument that is a tensor of matching shape.

<img src="https://miro.medium.com/max/1684/1*EWpoG5KayZSqkWmwM_wMFQ.png" alt="Computation Graph with Gradients" width="500"/>

In [ ]:
import torch

Create a tensor and set requires_grad=True to track computation with it



In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do an operation of tensor:



In [ ]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [ ]:
print(y.grad_fn)

Do more operations on y:



In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
flag in-place. The input flag defaults to ``False`` if not given.



In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


Gradients
---------
Let's backprop now
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(1))``.



In [ ]:
out.backward()

print gradients d(out)/dx




In [ ]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.  
We have that $o = \frac{1}{4}\sum_i z_i$,  
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.  
Therefore,  
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence  
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



You can do many crazy things with autograd!



In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([1592.6987, -405.7932,  388.8080], grad_fn=<MulBackward0>)


In [ ]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


You can also stop autograd from tracking history on Tensors
with ``.requires_grad=True`` by wrapping the code block in
``with torch.no_grad()``:



In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**Read Later:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd




Neural Networks
===============

Neural networks can be constructed using the ``torch.nn`` package.

Now that you had a glimpse of ``autograd``, ``nn`` depends on
``autograd`` to define models and differentiate them.  
An ``nn.Module`` contains layers, and a method ``forward(input)`` that
returns the ``output``.

For example, look at this network that classifies digit images:
![convnet](https://pytorch.org/tutorials/_images/mnist.png)

It is a simple feed-forward network. It takes the input, feeds it
through several layers one after the other, and then finally gives the
output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

Define the network
------------------

Let’s define this network:



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # kernel
        # 1 input channel, 6 output channels, 5x5 convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you
using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``



In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's weight

10
torch.Size([6, 1, 5, 5])


Let try a random 32 x 32 input:

Note: Expected input size to this net (LeNet) is 32 x 32. To use this net on
MNIST dataset, please resize the images from the dataset to 32 x 32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0066, -0.0514,  0.1422, -0.1322,  0.0855,  0.0106,  0.0483, -0.0117,
         -0.1043,  0.1359]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random
gradients:



In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

Note: ``torch.nn`` package only supports inputs that are in mini-batch form, but not a single sample.

For example, ``nn.Conv2d`` will take in a 4D Tensor of ``nSamples x nChannels x Height x Width``.

If you have a single sample, use ``input.unsqueeze(0)`` to add
a fake batch dimension.


Loss Function
-------------
A loss function takes the (output, target) pair as inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
[loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions) under the
nn package.  
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6725, grad_fn=<MseLossBackward>)


Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:
```
    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss
```

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients, otherwise gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0053, -0.0066,  0.0020,  0.0101,  0.0064, -0.0251])


Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

```
    weight = weight - learning_rate * gradient
```



We can implement this using simple python code:

```
    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)
``` 
Various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp are implemented in ``torch.optim`` package.
  
Gradient buffers had to be manually set to zero using ``optimizer.zero_grad()``.  





In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update